Please head over to this website to view the results: https://walterra.github.io/covid-19/

*I really appreciate if you're interested in the code, however, proceed with caution: All of this is a bunch of spaghetti code that gets the job done. There's lot of room to improve things and make the Python code more readable and simple. (It wouldn't change a lot about the generated VEGA specs though, they are fine, thanks Altair!*

In [1]:
import altair as alt
import json
import pandas as pd

In [2]:
counties = ['B', 'K', 'Noe', 'Ooe', 'S', 'St', 'T', 'V', 'W']
counties_confirmed = ['dataTime', 'B_confirmed', 'K_confirmed', 'Noe_confirmed', 'Ooe_confirmed', 'S_confirmed', 'St_confirmed', 'T_confirmed', 'V_confirmed', 'W_confirmed']
counties_death = ['dataTime', 'B_death', 'K_death', 'Noe_death', 'Ooe_death', 'S_death', 'St_death', 'T_death', 'V_death', 'W_death']
counties_recovered = ['dataTime', 'B_recovered', 'K_recovered', 'Noe_recovered', 'Ooe_recovered', 'S_recovered', 'St_recovered', 'T_recovered', 'V_recovered', 'W_recovered']



In [35]:
events = {
    'dataTime': ['16.03.2020 00:00:00'],
    'label': ['Lockdown'],
    'value': 500
}
df_events = pd.DataFrame.from_dict(events)
df_events['dataTime'] = pd.to_datetime(df_events['dataTime'], format='%d.%m.%Y %H:%M:%S')
df_events.head()

,dataTime,label,value
0,2020-03-16,Lockdown,500


In [3]:
df = pd.read_csv("../data/corin_data.csv")
# Use this line if only morning data is available and not the full day (removes last row)
# df = df[:-1]
df['dataTime'] = pd.to_datetime(df['dataTime'], format='%d.%m.%Y %H:%M:%S')
df_confirmed = df[counties_confirmed]
df_confirmed.tail()

,dataTime,B_confirmed,K_confirmed,Noe_confirmed,Ooe_confirmed,S_confirmed,St_confirmed,T_confirmed,V_confirmed,W_confirmed
128,2020-04-27 15:00:00,320.0,407.0,2573.0,2248.0,1214.0,1745.0,3484.0,868.0,2380.0
129,2020-04-28 08:00:00,321.0,407.0,2574.0,2250.0,1214.0,1747.0,3487.0,868.0,2388.0
130,2020-04-28 15:00:00,321.0,410.0,2577.0,2251.0,1214.0,1751.0,3494.0,869.0,2399.0
131,2020-04-29 08:00:00,321.0,410.0,2583.0,2251.0,1215.0,1755.0,3496.0,869.0,2423.0
132,2020-04-29 15:00:00,322.0,410.0,2591.0,2252.0,1215.0,1757.0,3496.0,870.0,2439.0


In [37]:
def c_line(att='total_confirmed',col='blue',scaleType='linear',label='value',highlight=False):
    the_base = alt.Chart(df_confirmed, width=180, height=120).transform_filter(
        alt.datum[att] > 0  
    ).transform_timeunit(
        date='yearmonthdate(dataTime)'
    ).transform_aggregate(
        max_att='max('+att+')',
        groupby=['date']
    ).transform_window(
        rolling_max='max(max_att)',
        rolling_min='min(max_att)',
        frame=[-1, 0]
    ).transform_calculate(
        diff='(datum.rolling_max - datum.rolling_min)'
    )
    
    the_chart = the_base.mark_line(
        color=col,
        size=2,
        strokeJoin='round'
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title='', axis=alt.Axis(minExtent=50))
    )

    if highlight is False:
        return the_chart

    the_tooltip = the_base.mark_circle(
        color=col,
        size=200,
        opacity=0
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title='', axis=alt.Axis(minExtent=50)),
        tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('diff:Q', title=label)]
    )
    
    the_highlight = the_base.transform_window(
        sort=[alt.SortField("date", order="descending")], 
        rank="rank(date)"
    ).transform_filter(
        alt.datum.rank == 1
    )
    
    the_text = the_highlight.mark_text(
        align='left',
        color=col,
        dx=5,
        width=500
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title=''),
        text=alt.Text("diff:Q")
    )
    
    # transparent text so every small multiple has the same right padding
    the_alibi_text = the_highlight.mark_text(
        align='left',
        color=col,
        dx=5,
        opacity=0
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title=''),
        text=alt.value('100000')
    )
    
    the_mark = the_highlight.mark_circle(
        color=col,
        opacity=1
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title=''),
    )
    
    rule_events = alt.Chart(df_events, width=180, height=120).mark_rule().encode(
        alt.Y('value:Q'),
        alt.X('dataTime:T')
    )
    
    text_events = alt.Chart(df_events, width=180, height=120).mark_text(
        align='left', dx=3, dy=5
    ).encode(
        alt.X('dataTime:T'),
        alt.Y('value:Q'),
        text='label',
        opacity=alt.value(0.5)
    )
    
    return the_chart + the_alibi_text + the_text + the_mark + the_tooltip

# ['B', 'K', 'Noe', 'Ooe', 'S', 'St', 'T', 'V', 'W']
def county_sm(highlight='T',title='Tirol'):
    highlight_color='#1f77b4'
    blur='#ddd'
    c_B = c_line('B_confirmed', highlight_color if highlight=='B' else blur, 'linear', 'Burgenland')
    c_K = c_line('K_confirmed', highlight_color if highlight=='K' else blur, 'linear', 'Kärnten')
    c_Noe = c_line('Noe_confirmed', highlight_color if highlight=='Noe' else blur, 'linear', 'Niederösterreich')
    c_Ooe = c_line('Ooe_confirmed', highlight_color if highlight=='Ooe' else blur, 'linear', 'Oberösterreich')
    c_S = c_line('S_confirmed', highlight_color if highlight=='S' else blur, 'linear', 'Salzburg')
    c_St = c_line('St_confirmed', highlight_color if highlight=='St' else blur, 'linear', 'Steiermark')
    c_T = c_line('T_confirmed', highlight_color if highlight=='T' else blur, 'linear', 'Tirol')
    c_V = c_line('V_confirmed', highlight_color if highlight=='V' else blur, 'linear', 'Vorarlberg')
    c_W = c_line('W_confirmed', highlight_color if highlight=='W' else blur, 'linear', 'Wien')
    c_highlight = c_line(highlight+'_confirmed', highlight_color, 'linear', title, True)

    return alt.layer(c_B, c_K, c_Noe, c_Ooe, c_S, c_St, c_T, c_V, c_W, c_highlight).properties(title=title)

chart_confirmed = (
    (county_sm('B', 'Burgenland') |
    county_sm('K', 'Kärnten') |
    county_sm('Noe', 'Niederösterreich')) &
    (county_sm('Ooe', 'Oberösterreich') |
    county_sm('S', 'Salzburg') |
    county_sm('St', 'Steiermark')) &
    (county_sm('T', 'Tirol') |
    county_sm('V', 'Vorarlberg') |
    county_sm('W', 'Wien'))
).configure_axis(
    grid=True,
    gridColor="#eee",
    domainColor="#ddd",
    tickColor="#ddd",
    labelColor="gray",
    labelBound=True,
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=12,
    fontWeight='normal',
#     anchor='start',
    color='gray'
)
    
chart_confirmed

alt.VConcatChart(...)

In [12]:
df_death = df[counties_death]

def c_line(att='total_confirmed',col='blue',scaleType='linear',label='value',highlight=False):
    the_base = alt.Chart(df_death, width=180, height=120).transform_filter(
        alt.datum[att] > 0  
    ).transform_timeunit(
        date='yearmonthdate(dataTime)'
    ).transform_aggregate(
        max_att='max('+att+')',
        groupby=['date']
    ).transform_window(
        rolling_max='max(max_att)',
        rolling_min='min(max_att)',
        frame=[-1, 0]
    ).transform_calculate(
        diff='(datum.rolling_max - datum.rolling_min)'
    )

    the_chart = the_base.mark_line(
        color=col,
        size=2,
        strokeJoin='round'
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title='', axis=alt.Axis(minExtent=50))
    )

    if highlight is False:
        return the_chart

    the_tooltip = the_base.mark_circle(
        color=col,
        size=200,
        opacity=0
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title='', axis=alt.Axis(minExtent=50)),
        tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('diff:Q', title=label)]
    )
    
    the_highlight = the_base.transform_window(
        sort=[alt.SortField("date", order="descending")], 
        rank="rank(date)"
    ).transform_filter(
        alt.datum.rank == 1
    )
    
    the_text = the_highlight.mark_text(
        align='left',
        color=col,
        dx=5,
        width=500
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title=''),
        text=alt.Text("diff:Q")
    )
    
    # transparent dummy text so every small multiple has the same right padding
    the_alibi_text = the_highlight.mark_text(
        align='left',
        color=col,
        dx=5,
        opacity=0
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title=''),
        text=alt.value('100000')
    )
    
    the_mark = the_highlight.mark_circle(
        color=col,
        opacity=1
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title=''),
    )
    
    return the_chart + the_alibi_text + the_text + the_mark + the_tooltip

# ['B', 'K', 'Noe', 'Ooe', 'S', 'St', 'T', 'V', 'W']
def county_sm(highlight='T',title='Tirol'):
    highlight_color='#d62728'
    blur='#ddd'
    c_B = c_line('B_death', highlight_color if highlight=='B' else blur, 'linear', 'Burgenland')
    c_K = c_line('K_death', highlight_color if highlight=='K' else blur, 'linear', 'Kärnten')
    c_Noe = c_line('Noe_death', highlight_color if highlight=='Noe' else blur, 'linear', 'Niederösterreich')
    c_Ooe = c_line('Ooe_death', highlight_color if highlight=='Ooe' else blur, 'linear', 'Oberösterreich')
    c_S = c_line('S_death', highlight_color if highlight=='S' else blur, 'linear', 'Salzburg')
    c_St = c_line('St_death', highlight_color if highlight=='St' else blur, 'linear', 'Steiermark')
    c_T = c_line('T_death', highlight_color if highlight=='T' else blur, 'linear', 'Tirol')
    c_V = c_line('V_death', highlight_color if highlight=='V' else blur, 'linear', 'Vorarlberg')
    c_W = c_line('W_death', highlight_color if highlight=='W' else blur, 'linear', 'Wien')
    c_highlight = c_line(highlight+'_death', highlight_color, 'linear', title, True)

    return alt.layer(c_B, c_K, c_Noe, c_Ooe, c_S, c_St, c_T, c_V, c_W, c_highlight).properties(title=title)

chart_death = (
    (county_sm('B', 'Burgenland') |
    county_sm('K', 'Kärnten') |
    county_sm('Noe', 'Niederösterreich')) &
    (county_sm('Ooe', 'Oberösterreich') |
    county_sm('S', 'Salzburg') |
    county_sm('St', 'Steiermark')) &
    (county_sm('T', 'Tirol') |
    county_sm('V', 'Vorarlberg') |
    county_sm('W', 'Wien'))
).configure_axis(
    grid=True,
    gridColor="#eee",
    domainColor="#ddd",
    tickColor="#ddd",
    labelColor="gray",
    labelBound=True,
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=12,
    fontWeight='normal',
#     anchor='start',
    color='gray'
)
    
chart_death

alt.VConcatChart(...)

In [13]:
df_recovered = df[counties_recovered]

def c_line(att='total_confirmed',col='blue',scaleType='linear',label='value',highlight=False):
    the_base = alt.Chart(df_recovered, width=180, height=120).transform_filter(
        alt.datum[att] > 0  
    ).transform_timeunit(
        date='yearmonthdate(dataTime)'
    ).transform_aggregate(
        max_att='max('+att+')',
        groupby=['date']
    ).transform_window(
        rolling_max='max(max_att)',
        rolling_min='min(max_att)',
        frame=[-1, 0]
    ).transform_calculate(
        diff='(datum.rolling_max - datum.rolling_min)'
    )

    the_chart = the_base.mark_line(
        color=col,
        size=2,
        strokeJoin='round'
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title='', axis=alt.Axis(minExtent=50))
    )

    if highlight is False:
        return the_chart

    the_tooltip = the_base.mark_circle(
        color=col,
        size=200,
        opacity=0
    ).encode(
        alt.X('date:T', title=''),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title='', axis=alt.Axis(minExtent=50)),
        tooltip=[alt.Tooltip('date:T', title='Datum'), alt.Tooltip('diff:Q', title=label)]
    )
    
    the_highlight = the_base.transform_window(
        sort=[alt.SortField("date", order="descending")], 
        rank="rank(date)"
    ).transform_filter(
        alt.datum.rank == 1
    )
    
    the_text = the_highlight.mark_text(
        align='left',
        color=col,
        dx=5,
        width=500
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title=''),
        text=alt.Text("diff:Q")
    )
    
    # transparent text so every small multiple has the same right padding
    the_alibi_text = the_highlight.mark_text(
        align='left',
        color=col,
        dx=5,
        opacity=0
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title=''),
        text=alt.value('100000')
    )
    
    the_mark = the_highlight.mark_circle(
        color=col,
        opacity=1
    ).encode(
        alt.X('date:T'),
        alt.Y('diff:Q', scale=alt.Scale(type=scaleType), title=''),
    )
    
    return the_chart + the_alibi_text + the_text + the_mark + the_tooltip

# ['B', 'K', 'Noe', 'Ooe', 'S', 'St', 'T', 'V', 'W']
def county_sm(highlight='T',title='Tirol'):
    highlight_color='#ff7f0e'
    blur='#ddd'
    c_B = c_line('B_recovered', highlight_color if highlight=='B' else blur, 'linear', 'Burgenland')
    c_K = c_line('K_recovered', highlight_color if highlight=='K' else blur, 'linear', 'Kärnten')
    c_Noe = c_line('Noe_recovered', highlight_color if highlight=='Noe' else blur, 'linear', 'Niederösterreich')
    c_Ooe = c_line('Ooe_recovered', highlight_color if highlight=='Ooe' else blur, 'linear', 'Oberösterreich')
    c_S = c_line('S_recovered', highlight_color if highlight=='S' else blur, 'linear', 'Salzburg')
    c_St = c_line('St_recovered', highlight_color if highlight=='St' else blur, 'linear', 'Steiermark')
    c_T = c_line('T_recovered', highlight_color if highlight=='T' else blur, 'linear', 'Tirol')
    c_V = c_line('V_recovered', highlight_color if highlight=='V' else blur, 'linear', 'Vorarlberg')
    c_W = c_line('W_recovered', highlight_color if highlight=='W' else blur, 'linear', 'Wien')
    c_highlight = c_line(highlight+'_recovered', highlight_color, 'linear', title, True)

    return alt.layer(c_B, c_K, c_Noe, c_Ooe, c_S, c_St, c_T, c_V, c_W, c_highlight).properties(title=title)

chart_recovered = (
    (county_sm('B', 'Burgenland') |
    county_sm('K', 'Kärnten') |
    county_sm('Noe', 'Niederösterreich')) &
    (county_sm('Ooe', 'Oberösterreich') |
    county_sm('S', 'Salzburg') |
    county_sm('St', 'Steiermark')) &
    (county_sm('T', 'Tirol') |
    county_sm('V', 'Vorarlberg') |
    county_sm('W', 'Wien'))
).configure_axis(
    grid=True,
    gridColor="#eee",
    domainColor="#ddd",
    tickColor="#ddd",
    labelColor="gray",
    labelBound=True,
).configure_view(
    strokeWidth=0
).configure_title(
    fontSize=12,
    fontWeight='normal',
#     anchor='start',
    color='gray'
)
    
chart_recovered

alt.VConcatChart(...)

In [7]:
# save chart_counties_confirmed as formatted JSON
with open('../data/chart_counties_confirmed_daily.json', 'w') as jsonFile:
    json.dump(json.loads(chart_confirmed.to_json()), jsonFile, indent=4, sort_keys=True)

In [8]:
# save chart_counties_death as formatted JSON
with open('../data/chart_counties_death_daily.json', 'w') as jsonFile:
    json.dump(json.loads(chart_death.to_json()), jsonFile, indent=4, sort_keys=True)

In [9]:
# save chart_counties_recovered as formatted JSON
with open('../data/chart_counties_recovered_daily.json', 'w') as jsonFile:
    json.dump(json.loads(chart_recovered.to_json()), jsonFile, indent=4, sort_keys=True)